In [1]:
import torch
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

from torchinfo import summary

import numpy as np
import time

In [2]:
IMAGENET_DIR = '/home/data/ImageNet'
BATCH_SIZE = 128
LOADER_WORKERS = 4

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
w = torchvision.models.efficientnet.EfficientNet_V2_L_Weights.IMAGENET1K_V1
model = torchvision.models.efficientnet_v2_l(weights=w)
model = model.to(device)
model = model.eval()

In [5]:
summary(model, (BATCH_SIZE, 3, 224, 224))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [128, 1000]               --
├─Sequential: 1-1                                       [128, 1280, 7, 7]         --
│    └─Conv2dNormActivation: 2-1                        [128, 32, 112, 112]       --
│    │    └─Conv2d: 3-1                                 [128, 32, 112, 112]       864
│    │    └─BatchNorm2d: 3-2                            [128, 32, 112, 112]       64
│    │    └─SiLU: 3-3                                   [128, 32, 112, 112]       --
│    └─Sequential: 2-2                                  [128, 32, 112, 112]       --
│    │    └─FusedMBConv: 3-4                            [128, 32, 112, 112]       9,280
│    │    └─FusedMBConv: 3-5                            [128, 32, 112, 112]       9,280
│    │    └─FusedMBConv: 3-6                            [128, 32, 112, 112]       9,280
│    │    └─FusedMBConv: 3-7                      

In [6]:
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
transform = torchvision.models.efficientnet.EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms()
transform

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.5, 0.5, 0.5]
    std=[0.5, 0.5, 0.5]
    interpolation=InterpolationMode.BICUBIC
)

In [7]:
dataset = torchvision.datasets.ImageNet(root=IMAGENET_DIR, transform=transform, split='val')
loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKERS)

In [8]:
n_top1 = 0
n_top5 = 0
cnt = 0

s = time.time()
pred_tms = []
with torch.no_grad():
    for images, labels in loader:

        x = images.to(device)
        y = labels.to(device)
        
        ss = time.time()
        output = model(x)
        pred_tms.append(time.time()-ss)
        
        cnt += output.size(0)
        
        _, pred_top1 = output.max(1)
        _, pred_top5 = output.topk(5, 1, True, True)
        
        n_top1 += torch.eq(pred_top1, y).sum().item()
        n_top5 += torch.isin(pred_top5, y).max(1)[0].sum().item()

        print(f"\rstep: {cnt}/{len(dataset)}", end='')
total_tm = time.time() - s
print()
print(f"top-1:  {n_top1/cnt:0.4f}")
print(f"top-5:  {n_top5/cnt:0.4f}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Total Time: {total_tm:0.4f} ({total_tm/len(dataset):0.4f})")
print(f"Average Prediction Time: {np.mean(pred_tms):0.4f}")

step: 50000/50000
top-1:  0.8580
top-5:  0.9812
Batch Size: 128
Total Time: 493.7320 (0.0099)
Average Prediction Time: 0.5770
